# Spaceship Titanic Competition

*Created by: Taylor Daugherty*

*Created on: 5/23/2023    Last updated: 6/13/2023 - Clean up notebook*

This notebook is used to practice using cross validation.


**Input File:** train.csv and test.csv from the spaceship-titanic competition

- train.csv contains a training set of data with features and true values

- test.csv contains a testing set of data with only features. This is what will be used to make predictions and submit to the competition


**Purpose of notebook:** Gain more practice with cross validation and make submissions to the spaceship titanic competition

## Table of Contents

1. Import packages

2. Useful functions

3. Initial Model

4. Improvement 1: Remove 'Converted' group

5. Improvement 2: Impute NaN values

    a. Constant
    
    b. Mean
    
    c. Median

## Import packages

Import the packages necessary to run the notebook

1. numpy: used for linear algebra

2. pandas: used for dataframe creation and manipulation

4. `SimpleImputer`: Used to fill NaN values

5. `LogisticRegression`: a classification model using logistic regression

6. `KNeighborsClassifier`: a classification model using KNN

7. `SVC`: a classification model using SVM

8. `KFold` and `cross_val_score`: used for cross validation

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold, cross_val_score

### File paths

These are the filepaths to the original data

In [2]:
train_filepath = '/kaggle/input/spaceship-titanic/train.csv'
test_filepath = '/kaggle/input/spaceship-titanic/test.csv'

In [3]:
df_train = pd.read_csv(train_filepath)
df_test = pd.read_csv(test_filepath)

### Fill Lists

There are many features with missing values in the dataset. For this reason, almost every column needs to fill missing values.

These two lists are designed to identify features and common values to fill the NaN values with.

- The first variable is a list of feature names that have missing values and will need to be filled

- The second variable is a list of the most frequent values for their respective columns

In [4]:
fill_features = ['HomePlanet', 'Destination', 'Side', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']
fill_freq = ['Earth', 'TRAPPIST-1e', 'S', False, False]

### Mapping Lists

There are many columns in the dataset that contain purely categorical data. Categorical data cannot be used in machine learning models, so the data must be converted to numbers

These two lists are designed to identify features and common values to map the categorical data.

- The first variable is a list of feature names that have categorical values and will need to be mapped

- The second variable is a list of dictionaries to use in mapping. The mapping values are based on frequency in the dataset with higher numbers corresponding to higher frequency

In [5]:
map_features = ['HomePlanet','Destination','Deck','Side']

map_dictionaries = [{'Earth':2, 'Europa':1, 'Mars':0}, 
                    {'TRAPPIST-1e':2, '55 Cancri e':1, 'PSO J318.5-22':0}, 
                    {'T':0, 'A':1, 'D':2, 'C':3, 'B':4, 'E':5, 'G':6, 'F':7}, 
                    {'P':0, 'S':1}]

### List of models

These are the models that will be used during cross-validation. 

**Models:**

- Logistic Regression

- KNN

- SVM

In [6]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))

In [7]:
rs_mean = df_train['RoomService'].mean()
fc_mean = df_train['FoodCourt'].mean()
sm_mean = df_train['ShoppingMall'].mean()
spa_mean = df_train['Spa'].mean()
vr_mean = df_train['VRDeck'].mean()
age_mean = df_train['Age'].mean()

rs_test_mean = df_test['RoomService'].mean()
fc_test_mean = df_test['FoodCourt'].mean()
sm_test_mean = df_test['ShoppingMall'].mean()
spa_test_mean = df_test['Spa'].mean()
vr_test_mean = df_test['VRDeck'].mean()
age_test_mean = df_test['Age'].mean()

mean_nan_fill_train = [rs_mean, fc_mean, sm_mean, spa_mean, vr_mean, age_mean]
mean_nan_fill_test = [rs_test_mean, fc_test_mean, sm_test_mean, spa_test_mean, vr_test_mean, age_test_mean]

In [8]:
rs_med = df_train['RoomService'].median()
fc_med = df_train['FoodCourt'].median()
sm_med = df_train['ShoppingMall'].median()
spa_med = df_train['Spa'].median()
vr_med = df_train['VRDeck'].median()
age_med = df_train['Age'].median()

rs_test_med = df_test['RoomService'].median()
fc_test_med = df_test['FoodCourt'].median()
sm_test_med = df_test['ShoppingMall'].median()
spa_test_med = df_test['Spa'].median()
vr_test_med = df_test['VRDeck'].median()
age_test_med = df_test['Age'].median()

med_nan_fill_train = [rs_med, fc_med, sm_med, spa_med, vr_med, age_med]
med_nan_fill_test = [rs_test_med, fc_test_med, sm_test_med, spa_test_med, vr_test_med, age_test_med]

### perform_cross_validation()

This function goes through the steps to perform Stratified K-fold cross validation using the list of models described above.

**Input:** A dataframe containing the features use to build the model, a Series of the true values associated with the feature list

**Output:** Printed result for the mean and standard deviation of each model

In [9]:
def perform_cross_validation(X_train, y_train):
    '''
    This function goes through the steps to perform Stratified K-fold cross validation using the list of models described above.
    
    Input: 
        - A dataframe containing the features use to build the model
        - A Series of the true values associated with the feature list
    
    Output: Printed result for the mean and standard deviation of each model
    '''
    results = dict()

    for name, model in models:
        kfold = StratifiedKFold(n_splits=10)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
        results[name] = (cv_results.mean(), cv_results.std())

    print('Model\t\tCV Mean\t\tCV std')
    print(results)

### expand_cabin()

This expands the 'Cabin' column into the 'Deck' and 'Side'. 

The 'Num' feature is removed because there are two many unique values. The 'Cabin' feature is removed since it is not needed after the expansion

**Input:** A dataframe to with the Cabin feature to expand

**Output:** A new dataframe with the cabin expanded to 'Deck' and 'Side' and without 'Cabin' or 'Num'

In [10]:
def expand_cabin(df):
    '''
    This expands the 'Cabin' column into the 'Deck' and 'Side'.
    
    Input: A dataframe to with the Cabin feature to expand
    
    Output: A new dataframe with the cabin expanded to 'Deck' and 'Side' and without 'Cabin' or 'Num'
    '''
    df[['Deck','Num','Side']] = df['Cabin'].astype(str).str.split('/', expand=True)
    df = df.drop('Cabin', axis=1)
    df = df.drop('Num', axis=1)
    return df

### fill_na()

Fill the NaN values of the features with the fill values

**Input:** A dataframe, a list of features, a list of fill values

- Dataframe to fill the missing values in

- A list of features to look for missing values

- A list of values to fill the missing values of the feature with

**Output:** A dataframe with the found missing values filled

In [11]:
def fill_na(df, features, fill_vals):
    '''
    Fill the NaN values of the features with the fill values
    
    Input:
        - Dataframe to fill the missing values in
        - A list of features to look for missing values
        - A list of values to fill the missing values of the feature with
    
    Output: A dataframe with the found missing values filled
    '''
    df_filled = df
    
    for i in range(len(features)):
        feature = features[i]
        fill_val = fill_vals[i]
        
        df_filled[feature] = df_filled[feature].fillna(fill_val)
        
    return df_filled

### map_cat()

Map categorical data to be numeric

**Input:** A dataframe, a list of features, a list of dictionaries

- Dataframe for the mapping to take place in

- List of features that contain categorical data

- List of dictionaries to govern how to map the feature

**Output:** A dataframe with the given features mapped according to the dictionaries

In [12]:
def map_cat(df, features, dictionaries):
    '''
    Map categorical data to be numeric
    
    Input:
        - Dataframe for the mapping to take place in
        - List of features that contain categorical data
        - List of dictionaries to govern how to map the feature
    
    Output: A dataframe with the given features mapped according to the dictionaries
        
    '''
    df_mapped = df
    
    for i in range(len(features)):
        feature = features[i]
        dictionary = dictionaries[i]
        
        df_mapped[feature] = df[feature].map(dictionary)
        
    return df_mapped

### normalize()

Normalizes a Series

**Input:** A feature of type Series

**Output:** The normalized feature of type Series

In [13]:
def normalize(feature):
    '''
    This function normalizes a Series
    
    Input: A feature of type Series
    
    Output: The normalized feature of type Series
    '''
    return (feature - feature.mean())/feature.std()

### normalize_features()

Normalizes all features in a given dataframe. This will normalize ALL features, so ensure that the inputted dataframe consists only of numeric values.

**Input:** A dataframe to normalize

**Output:** A normalized dataframe

In [14]:
def normalize_features(df):
    '''
    This function normalizes all features in a dataframe
    
    Input: A pandas dataframe
    
    Output: The normalized dataframe
    '''
    for column in df.columns:
        df[column] = normalize(df[column])
    return df

In [15]:
def prepare_dataframe(df, fill_nans):
    df = df.drop('Name', axis=1)
    df = expand_cabin(df)
    df['Deck'] = df['Deck'].replace('nan','F')
    
    fill_vals = fill_freq + fill_nans
    df_filled = fill_na(df, fill_features, fill_vals)
    
    df_mapped = map_cat(df_filled, map_features, map_dictionaries)
    return df_mapped

In [16]:
# df_tr = pd.read_csv(train_filepath, index_col='PassengerId')
# df_tr = prepare_dataframe(df_tr, mean_nan_fill_train)

# X = df_tr.drop('Transported', axis=1)
# X_norm = normalize(X)
# X_norm.head()

**--------------------------------------------------------------------------------**

## Fill with Mean

This model fills the missing values for spending features with the mean of the feature

**Unique attributes:**

- Fill NaN with feature mean

**Result of CV:** The model used to make predictions was SVM because it had the highest accuracy score

**Accuracy:** The accuracy after submitting the predictions

    0.79541

### Prepare dataframe

Perform all necessary preprocessing steps for cross-validation

**Steps:**

1. Load original dataset

2. Adjust features

3. 

In [17]:
df_train = pd.read_csv(train_filepath, index_col='PassengerId')
df_test = pd.read_csv(test_filepath, index_col='PassengerId')

In [18]:
df_train.head(10)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True


In [19]:
df_train = df_train.drop('Name', axis=1)
df_test = df_test.drop('Name', axis=1)

In [20]:
df_train = expand_cabin(df_train)
df_test = expand_cabin(df_test)

In [21]:
df_train['Deck'] = df_train['Deck'].replace('nan','F')
df_test['Deck'] = df_test['Deck'].replace('nan','F')

In [22]:
X = df_train.drop('Transported', axis=1)

y = df_train['Transported']

In [23]:
rs_mean = X['RoomService'].mean()
fc_mean = X['FoodCourt'].mean()
sm_mean = X['ShoppingMall'].mean()
spa_mean = X['Spa'].mean()
vr_mean = X['VRDeck'].mean()
age_mean = X['Age'].mean()

rs_test_mean = df_test['RoomService'].mean()
fc_test_mean = df_test['FoodCourt'].mean()
sm_test_mean = df_test['ShoppingMall'].mean()
spa_test_mean = df_test['Spa'].mean()
vr_test_mean = df_test['VRDeck'].mean()
age_test_mean = df_test['Age'].mean()

In [24]:
fill_features = ['Deck','HomePlanet', 'Destination', 'Side', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']
fill_vals = ['F', 'Earth', 'TRAPPIST-1e', 'S', False, False]

fill_vals_train = fill_vals + [rs_mean, fc_mean, sm_mean, spa_mean, vr_mean, age_mean]
fill_vals_test = fill_vals + [rs_test_mean, fc_test_mean, sm_test_mean, spa_test_mean, vr_test_mean, age_test_mean]

In [25]:
X_imputed = fill_na(X, fill_features, fill_vals_train)
X_test_imputed = fill_na(df_test, fill_features, fill_vals_test)
X_imputed.head(10)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.000000,B,P
0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.000000,F,S
0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.000000,A,S
0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.000000,A,S
0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.000000,F,S
0005_01,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.000000,F,P
0006_01,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.000000,F,S
0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,304.854791,G,S
0007_01,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.000000,F,S


In [26]:
map_features = ['HomePlanet','Destination','Deck','Side']

map_dictionaries = [{'Earth':2, 'Europa':1, 'Mars':0}, 
                    {'TRAPPIST-1e':2, '55 Cancri e':1, 'PSO J318.5-22':0}, 
                    {'T':0, 'A':1, 'D':2, 'C':3, 'B':4, 'E':5, 'G':6, 'F':7}, 
                    {'P':0, 'S':1}]

In [27]:
X_mapped = map_cat(X_imputed, map_features, map_dictionaries)
X_test_mapped = map_cat(X_test_imputed, map_features, map_dictionaries)

X_mapped.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,1,False,2,39.0,False,0.0,0.0,0.0,0.0,0.0,4,0
0002_01,2,False,2,24.0,False,109.0,9.0,25.0,549.0,44.0,7,1
0003_01,1,False,2,58.0,True,43.0,3576.0,0.0,6715.0,49.0,1,1
0003_02,1,False,2,33.0,False,0.0,1283.0,371.0,3329.0,193.0,1,1
0004_01,2,False,2,16.0,False,303.0,70.0,151.0,565.0,2.0,7,1


In [28]:
X_mapped.head(20)

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,1,False,2,39.0,False,0.0,0.0,0.000000,0.0,0.000000,4,0
0002_01,2,False,2,24.0,False,109.0,9.0,25.000000,549.0,44.000000,7,1
0003_01,1,False,2,58.0,True,43.0,3576.0,0.000000,6715.0,49.000000,1,1
0003_02,1,False,2,33.0,False,0.0,1283.0,371.000000,3329.0,193.000000,1,1
0004_01,2,False,2,16.0,False,303.0,70.0,151.000000,565.0,2.000000,7,1
0005_01,2,False,0,44.0,False,0.0,483.0,0.000000,291.0,0.000000,7,0
0006_01,2,False,2,26.0,False,42.0,1539.0,3.000000,0.0,0.000000,7,1
0006_02,2,True,2,28.0,False,0.0,0.0,0.000000,0.0,304.854791,6,1
0007_01,2,False,2,35.0,False,0.0,785.0,17.000000,216.0,0.000000,7,1


In [29]:
X_normalized = normalize_features(X)
X_test_normalized = normalize_features(X_test_mapped)

In [30]:
X_normalized.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Side
PassengerId,,,,,,,,,,,,
0001_01,-0.440360,-0.732728,0.601283,0.709396,-0.153054,-0.340570,-0.287298,-0.290800,-0.276648,-0.269007,-0.843795,-1.032805
0002_01,0.817212,-0.732728,0.601283,-0.336698,-0.153054,-0.175354,-0.281653,-0.248954,0.211493,-0.230181,0.919170,0.968125
0003_01,-0.440360,-0.732728,0.601283,2.034449,6.532879,-0.275393,1.955503,-0.290800,5.693962,-0.225769,-2.606761,0.968125
0003_02,-0.440360,-0.732728,0.601283,0.290958,-0.153054,-0.340570,0.517376,0.330206,2.683316,-0.098702,-2.606761,0.968125
0004_01,0.817212,-0.732728,0.601283,-0.894615,-0.153054,0.118702,-0.243395,-0.038046,0.225719,-0.267242,0.919170,0.968125


In [31]:
# perform_cross_validation(X_normalized, y)

In [32]:
X_normalized.isnull().sum()

HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Deck            0
Side            0
dtype: int64

In [33]:
clf = SVC().fit(X_normalized, y)

predictions = clf.predict(X_test_normalized)

submission1 = pd.DataFrame(data={'Transported':predictions}, index=df_test.index)

submission1.head()

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True


In [34]:
submission1.to_csv('2023-6-13_Submission1.csv')

Result: 0.79541

**--------------------------------------------------------------------------------**

In [35]:
df_train2 = pd.read_csv(train_filepath, index_col='PassengerId')
df_test2 = pd.read_csv(test_filepath, index_col='PassengerId')

In [36]:
df_train2 = df_train2.drop('Name', axis=1)
df_test2 = df_test2.drop('Name', axis=1)

In [37]:
df_train2 = expand_cabin(df_train2)
df_test2 = expand_cabin(df_test2)

In [38]:
X2 = df_train2.drop('Transported', axis=1)

y2 = df_train2['Transported']

In [39]:
X2['Deck'] = X2['Deck'].replace('nan','F')
df_test2['Deck'] = df_test2['Deck'].replace('nan','F')

In [40]:
rs_med = X2['RoomService'].median()
fc_med = X2['FoodCourt'].median()
sm_med = X2['ShoppingMall'].median()
spa_med = X2['Spa'].median()
vr_med = X2['VRDeck'].median()
age_med = X2['Age'].median()

rs_test_med = df_test2['RoomService'].median()
fc_test_med = df_test2['FoodCourt'].median()
sm_test_med = df_test2['ShoppingMall'].median()
spa_test_med = df_test2['Spa'].median()
vr_test_med = df_test2['VRDeck'].median()
age_test_med = df_test2['Age'].median()

In [41]:
fill_features = ['HomePlanet', 'Destination', 'Side', 'CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']
fill_vals = ['Earth', 'TRAPPIST-1e', 'S', False, False]

fill_vals_train_med = fill_vals + [rs_med, fc_med, sm_med, spa_med, vr_med, age_med]
fill_vals_test_med = fill_vals + [rs_test_med, fc_test_med, sm_test_med, spa_test_med, vr_test_med, age_test_med]

In [42]:
X_imputed2 = fill_na(X2, fill_features, fill_vals_train_med)
X_test_imputed2 = fill_na(df_test2, fill_features, fill_vals_test_med)

In [43]:
map_features = ['HomePlanet','Destination','Deck','Side']

map_dictionaries = [{'Earth':2, 'Europa':1, 'Mars':0}, 
                    {'TRAPPIST-1e':2, '55 Cancri e':1, 'PSO J318.5-22':0}, 
                    {'T':0, 'A':1, 'D':2, 'C':3, 'B':4, 'E':5, 'G':6, 'F':7}, 
                    {'P':0, 'S':1}]

In [44]:
X_mapped2 = map_cat(X_imputed2, map_features, map_dictionaries)
X_test_mapped2 = map_cat(X_test_imputed2, map_features, map_dictionaries)

In [45]:
X_normalized2 = normalize_features(X_mapped2)
X_test_normalized2 = normalize_features(X_test_mapped2)

In [46]:
# perform_cross_validation(X_normalized2, y2)

In [47]:
clf_med = SVC().fit(X_normalized2, y2)

predictions_med = clf.predict(X_test_normalized2)

submission2 = pd.DataFrame(data={'Transported':predictions_med}, index=df_test.index)

submission2.head()

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True


In [48]:
submission2.to_csv('2023-6-13_Submission2.csv')

Result: 0.79611